In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def returnPoint(self):
        return (self.x, self.y)     

In [ ]:
class InnerNode:
    def __init__(self, idd = None, f = None, lson = None, rson = None, maxY = None, Q = None, B = 0):
        self.idd = idd #Debugging
        self.f = f #pointer to father
        self.lson = lson #pointer to left son
        self.rson = rson #pointer to right son
        self.maxY = maxY 
        #largest y-value in subtree of lson for lc
        #largest y-value in subtree of rson for rc    
        self.Q = Q #segment of Q_a that did not contribute to Q_f
        self.B = B #number of points of Q_a that contribute to Q_f
    
    def printData(self):
        printID = self.idd
        if self.f:
            printF = self.f.idd
        else:
            printF = "NONE"
        
        if(not self.lson is None):
            printLson = self.lson.idd
        else:
            printLson = "NONE"
            
        if(not self.rson is None):
            printRson = self.rson.idd
        else:
            printRson = "NONE"
        
        printMaxY = self.maxY
        if self.Q:
            printQ = "["
            for i in range(len(self.Q)-1):
                printQ += "({},{}),".format(self.Q[i].x,self.Q[i].y)
            printQ += "({},{})".format(self.Q[len(self.Q)-1].x, self.Q[len(self.Q)-1].y)
            printQ += "]"
        else:
            printQ = "NONE"
        if self.B:
            printB = self.B
        else:
            printB = "0"
            
        print("id: {}, f: {}, lson: {}, rson: {}, maxY: {}, Q: {}, B: {}".format(printID, printF, printLson, printRson, printMaxY, printQ, printB))
    

In [ ]:
import math
def create_points_circle(n):
    r = n*1000
    pi = math.pi
    points = []
    for i in range(n):
        x = math.cos(2*pi*(np.random.rand()*max)/n*i)*r *(np.random.rand())
        y = math.sin(2*pi*(np.random.rand()*max)/n*i)*r *(np.random.rand())
        x = np.ceil(x+r)
        y = np.ceil(y+r)
        x = np.absolute(x)
        y = np.absolute(y)
        for j in range(len(points)):
            while(np.absolute(points[j].x - x) < 2):
                x = math.cos(2*pi*(np.random.rand()*max)/n*i)*r *(np.random.rand())
                x = np.ceil(x)
                x = np.absolute(x)
            while(np.absolute(points[j].y - y) <1):
                y = math.cos(2*pi*(np.random.rand()*max)/n*i)*r *(np.random.rand())
                y = np.ceil(y)
                y = np.absolute(y)
        
        p = Point(x, y)
        points.append(p)
    return points

In [ ]:
def plotPoints(points, Y0, showAngles=False, hull=None):
    n = len(points)
    if(showAngles == True):
        point1 = [Y0.x, Y0.y]
        for i in range(n):
            point2 = [points[i].x,points[i].y]
            x_values = [point1[0], point2[0]]
            y_values = [point1[1], point2[1]]
            plt.plot(x_values, y_values, 'b-', linewidth=0.1)
            plt.plot(point2[0], point2[1], 'b.')
    
    if hull:
        x0 = hull[0].x
        y0 = hull[0].y
        
        for p in hull[1:]:
            x1 = p.x
            y1 = p.y
            plt.plot([x0, x1], [y0, y1])
            x0, y0 = x1, y1
        
    
    for i in range(n):
        plt.plot(points[i].x, points[i].y, "b.")
    plt.plot(Y0.x, Y0.y, "go")
    plt.show()

In [ ]:
def printList(points):
    for i in range(len(points)):
        print(points[i].x, points[i].y)

In [ ]:
def isBetween(a, b, c):
    if(c.y <= a.y and c.y >= b.y):
        if(c.x <= a.x and c.x >= b.x):
            return True
        if(c.x >= a.x and c.x <= b.x):
            return True
    if(c.y >= a.y and c.y <= b.y):
        if(c.x <= a.x and c.x >= b.x):
            return True
        if(c.x >= a.x and c.x <= b.x):
            return True
    return False

In [ ]:
def isIntersecting(line1, line2, isInfinite=False, case9 = False, debug=False, hull="lc"):
    """ 
    Returns the point of intersection of the lines passing through a2,a1 and b2,b1.
    a1: [x, y] a point on the first line
    a2: [x, y] another point on the first line
    b1: [x, y] a point on the second line
    b2: [x, y] another point on the second line
    """
    a1 = np.array([line1[0][0], line1[0][1]])
    a2 = np.array([line1[1][0], line1[1][1]])
    b1 = np.array([line2[0][0], line2[0][1]])
    b2 = np.array([line2[1][0], line2[1][1]])
    
    s = np.vstack([a1,a2,b1,b2])        # s for stacked
    h = np.hstack((s, np.ones((4, 1)))) # h for homogeneous
    l1 = np.cross(h[0], h[1])           # get first line
    l2 = np.cross(h[2], h[3])           # get second line
    x, y, z = np.cross(l1, l2)          # point of intersection
    if z == 0:                          # lines are parallel
        return np.inf, np.inf
    
    #print("a1: {}|{}, a2: {}|{}, b1: {}|{}, b2:{}|{}, POINT: {}|{}".format(a1[0],a1[1],a2[0],a2[1],b1[0],b1[1],b2[0],b2[1], x/z, y/z))
    #print(x/z,y/z)
    
    
    if(isInfinite):
     #   print("CHECKING FOR INFINITE CASE {}".format(hull))
        if(hull == "lc"):
            if(x/z >= line2[0][0] or x/z >=line2[1][0]):
      #          print("IS INFINITE")
                return x/z,y/z
            else:
                return np.inf, np.inf
        else:
            if(x/z <= line2[0][0] or x/z <=line2[1][0]):
       #         print("IS INFINITE")
                return x/z,y/z
            else:
                return np.inf, np.inf
    
    if(isBetween(Point(b1[0],b1[1]), Point(b2[0], b2[1]), Point(x/z,y/z)) or case9):
        #print("IS BETWEEEN")
        return x/z, y/z
    
    return np.inf, np.inf

#p1 = array( [503.0, 1859.0] )
#p2 = array( [1385.0, 1591.0] )

#p3 = array( [1671.0, 1125.0] )
#p4 = array( [1932.0, 904.0] )

#print(isIntersecting([p1,p2], [p3,p4]))
#print(seg_intersect( p1,p2, p3,p4))

#p1 = array( [2.0, 2.0] )
#p2 = array( [4.0, 3.0] )

#p3 = array( [6.0, 0.0] )
#p4 = array( [6.0, 3.0] )

#print(seg_intersect( p1,p2, p3,p4))

In [ ]:
### ALL DRAWING STUFF

def drawQStructure(Q, ax):
    x0 = Q[0].x
    y0 = Q[0].y
    for p in Q:
        
        x1 = p.x
        y1 = p.y
    
    for i in range(len(Q)):
        ax.plot(Q[i].x, Q[i].y, "b.")
        

def drawIntersection(p,q,a,b, ax):
    ax.plot([p[0], q[0]], [p[1], q[1]], 'k-', linewidth=0.5, alpha=0.7)
    ax.plot([a[0], b[0]], [a[1], b[1]], 'g-', linewidth=0.5)

def drawEdges(Q, ax):
    x0 = Q[0].x
    y0 = Q[0].y
    for p in Q:
            x1 = p.x
            y1 = p.y
            ax.plot([x0, x1], [y0, y1])
            x0, y0 = x1, y1
                   
def drawPQ(p,q,ax):
    ax.plot(p[0], p[1], "ro")
    ax.plot(q[0], q[1], "ro")
        
def drawPointColor(x, y, col, ax):
    ax.plot(x, y, col)   
        
def correctPlot(fig):
    n = len(fig.axes)
    for i in range(n):
        fig.axes[i].change_geometry(1, n+1, i+1)
    return fig

In [ ]:
def scale(p1,p2,factor):
    t0=0.5*(1.0-factor)
    t1=0.5*(1.0+factor)
    x1 = p1[0] +(p2[0] - p1[0]) * t0
    y1 = p1[1] +(p2[1] - p1[1]) * t0
    x2 = p1[0] +(p2[0] - p1[0]) * t1
    y2 = p1[1] +(p2[1] - p1[1]) * t1
    return [x1,y1],[x2,y2]

In [ ]:
def checkDuplicate(p1,p2, debug=False):
    if(p1[0] - p2[0] == 0 and p1[1] - p2[1] == 0):
        return True
    return False

In [ ]:
def newIsIntersecting(POI, succ, pred, slope, b, hull="lc"): #POI = Point Of Interest
    #f(x) = m*x + b
    # b = POI.y
    # m = slope
    # x = succ.x
    
    # f(y) = m*y + b
    # m = slope = POI1.y - POI2.y
    # b = POI.x
    # y = succ.y
    
    # 0 = NO second intersection
    # 1 = Second intersection above POI
    # 2 = Second intersection below POI
    
    #slope = (y2-y1)/(x2-x1) -> (x2-x1)/(y2-y1)
    
    if(pred is None and succ is None):
        return 0
    
    if(hull == "lc"): #lc-hull
        #succ is None    
        if(succ is None):
            if(pred[0] < (pred[1]-b)/slope):
                return 1
            if(pred[0] > (pred[1]-b)/slope):
                return 0
        #pred is None    
        if(pred is None):
            if(succ[0] < (succ[1]-b)/slope):
                return 2
            if(succ[0] > (succ[1]-b)/slope):
                return 0
        #No second intersection
        if((succ[0] > (succ[1]-b)/slope) and pred[0] > (pred[1]-b)/slope):
            return 0
        #One intersection above POI
        if((succ[0] > (succ[1]-b)/slope) and pred[0] < (pred[1]-b)/slope):
            return 1
         #One intersection below POI
        if((succ[0] < (succ[1]-b)/slope) and pred[0] > (pred[1]-b)/slope):
            return 2
        
    elif(hull == "rc"): #rc-hull
        
        #succ is None    
        if(succ is None):
            if(pred[0] > (pred[1]-b)/slope):#(slope*pred[1] + b)):
                return 1
            if(pred[0] < (pred[1]-b)/slope):
                return 0
        #pred is None    
        if(pred is None):
            if(succ[0] > (succ[1]-b)/slope):
                return 2
            if(succ[0] < (succ[1]-b)/slope):
                return 0
        #No second intersection
        if((succ[0] < (succ[1]-b)/slope) and pred[0] < (pred[1]-b)/slope):
            return 0
        #One intersection above POI
        if((succ[0] < (succ[1]-b)/slope) and pred[0] > (pred[1]-b)/slope):
            return 1
         #One intersection below POI
        if((succ[0] > (succ[1]-b)/slope) and pred[0] < (pred[1]-b)/slope):
            return 2  
    
    
   

    
    

In [ ]:
def findBridge(Q_A_IN, Q_C_IN, iteration, draw=True, debug=False, hull="lc"):

    Q_A = Q_A_IN.copy()
    Q_C = Q_C_IN.copy()
    
    
    if(len(Q_A) < 2):
        pID = 0
    else:    
        pID = math.floor(len(Q_A)/2)
    
    if(len(Q_C) < 2):
        qID = 0
    else:
        qID = math.floor(len(Q_C)/2)
    
    p = [Q_A[pID].x, Q_A[pID].y]
    q = [Q_C[qID].x, Q_C[qID].y]
     
#### CASE DETERMINATION ####

    p = [Q_A[pID].x, Q_A[pID].y]
    q = [Q_C[qID].x, Q_C[qID].y]
    
    if(len(Q_A) > 1):
        try:
            predA = [Q_A[pID-1].x, Q_A[pID-1].y]
        except IndexError:
            predA = None
    else:
        predA = None
    
    try:
        succA = [Q_A[pID+1].x, Q_A[pID+1].y]
    except IndexError:
        succA = None
    
    if(len(Q_C) > 1):
        try:
            predC = [Q_C[qID-1].x, Q_C[qID-1].y]
        except IndexError:
            predC = None
    else:
        predC = None
    
    try:
        succC = [Q_C[qID+1].x, Q_C[qID+1].y]
    except IndexError:
        succC = None
    
    slope = (p[1] - q[1])/(p[0] - q[0])
    b = ((q[0]*p[1])-(p[0]*q[1]))/(q[0]-p[0])
    
    #for i in range(len(Q_A)):
    #    print("Q_A ",i,Q_A[i].y)
   # 
   # for i in range(len(Q_C)):
   #     print("Q_C ",i,Q_C[i].y)
   # 
   # print("p: {}, q: {}".format(p,q))
   # print("predA: {}, succA: {}".format(predA, succA))
   # print("predC: {}, succC: {}".format(predC, succC))
   # print("slope: ",slope)
   # print("B: ", b)
    
    
    
    
    # 0 = NO second intersection
    # 1 = Second intersection above POI
    # 2 = Second intersection below POI
    
    InterA = newIsIntersecting(p, succA, predA, slope, b, hull=hull)
    InterC = newIsIntersecting(q, succC, predC, slope, b, hull=hull)
    
    #print("InterA: {}, InterC: {}".format(InterA,InterC))
    
    
    
    
    if(InterA == 0 and InterC == 0): #Case 1, return p,q immediately
        #print("case 1")
        return p, q
        
    
    if(InterA == 0 and InterC == 2):
        #case 2
        if(pID == len(Q_A)-1 and len(Q_A) > 1 and False):
            Q_A.pop(pID)
        else:
            for i in range(len(Q_A)-1, pID, -1):
                Q_A.pop(i)       

        if(qID == 0 and len(Q_C) > 1):
            Q_C.pop(qID)
        else:
            for i in range(qID-1, -1, -1):
                Q_C.pop(i)
        return findBridge(Q_A, Q_C, iteration+1, debug=debug, hull=hull)
        
        
    if(InterA == 0 and InterC == 1):
        #case 3
        if(pID == len(Q_A)-1 and len(Q_A) > 1 and False):
            Q_A.pop(pID)
        else:
            for i in range(len(Q_A)-1, pID, -1):
                Q_A.pop(i) 

        if(qID == len(Q_C)-1 and len(Q_C) > 1):
            Q_C.pop(qID)
        else:
            for i in range(len(Q_C)-1, qID-1, -1):
                Q_C.pop(i)
        return findBridge(Q_A, Q_C, iteration+1, debug=debug, hull=hull)
        
    if(InterA == 1 and InterC == 0):
        #case 4
        if(pID == len(Q_A)-1 and len(Q_A) > 1):
            Q_A.pop(pID)
        else:
            for i in range(len(Q_A)-1, pID, -1):
                Q_A.pop(i)       
        #for i in range(qID+1, -1, -1):
        if(qID == 0 and len(Q_C) > 1 and False):
            Q_C.pop(qID)
        else:
            for i in range(qID-1, -1, -1):
                Q_C.pop(i)
        return findBridge(Q_A, Q_C, iteration+1, debug=debug, hull=hull)
    
    if(InterA == 2 and InterC == 0):
        #case 5
        if(pID == 0 and len(Q_A) > 1):
            Q_A.pop(pID)
        else: 
            for i in range(pID-1, -1, -1):
                Q_A.pop(i)      

        if(qID == 0 and len(Q_C) > 1 and False):
            Q_C.pop(qID)
        else:
            for i in range(qID-1, -1, -1):
                Q_C.pop(i)
        return findBridge(Q_A, Q_C, iteration+1, debug=debug, hull=hull)

    if(InterA == 1 and InterC == 2):
        #case 6
        if(pID == len(Q_A)-1 and len(Q_A) > 1):
            Q_A.pop(pID)
        else:                
            for i in range(len(Q_A)-1, pID, -1):
                Q_A.pop(i)        
        if(qID == 0 and len(Q_C) > 1):
            Q_C.pop(qID)
        else:
            for i in range(qID-1, -1, -1):
                Q_C.pop(i)
        return findBridge(Q_A, Q_C, iteration+1, debug=debug, hull=hull)

    if(InterA == 2 and InterC == 2):
        #Case 7
        if(len(Q_C) == 1 and len(Q_A) > 1):
            Q_A.pop(pID)
        else:
            if(qID == 0 and len(Q_C) > 1):
                Q_C.pop(qID)
            else:
                for i in range(qID-1, -1, -1):
                    Q_C.pop(i)
        return findBridge(Q_A, Q_C, iteration+1, debug=debug, hull=hull)

    if(InterA == 1 and InterC == 1):
        #Case 8
        if(len(Q_A) == 1 and len(Q_C) > 1):
            Q_C.pop(qID)
        else:
            if(pID == len(Q_A)-1):
                Q_A.pop(pID)
            else:
                for i in range(len(Q_A)-1, pID-1, -1):
                    Q_A.pop(i)       
        return findBridge(Q_A, Q_C, iteration+1, debug=debug, hull=hull)
    
    if(InterA == 2 and InterC == 1):
        #Case 9, check for 9a or 9b
        #Tangent of A
        p1 = (Q_A[pID].x, Q_A[pID].y)
        p2 = (Q_A[pID-1].x, Q_A[pID-1].y)

        #Tangent of C
        q1 = (Q_C[qID].x, Q_C[qID].y)
        q2 = (Q_C[qID-1].x, Q_C[qID-1].y)

        #CenterLine
        yOfQ_A = Q_A[len(Q_A)-1].y
        yOfQ_C = Q_C[0].y

        centerline = (yOfQ_A + yOfQ_C)/2
        intersect = isIntersecting((p1,p2),(q1,q2))

        if(intersect[1] <= centerline): #Case 9a, remove after q on C           
            #CASE 9a Intersection is below 
            #print("case 9a")
            if(qID == len(Q_C)-1  and len(Q_C) > 1):
                Q_C.pop(qID)
            else:
                for i in range(len(Q_C)-1, qID, -1):
                    Q_C.pop(i) 
            #if(len(Q_A) > 1):
            #    Q_A.pop(pID)

        else: #Case 9b, remove before p on A
            #print("case 9b")
            #print("Q_A before")
            #for i in range(len(Q_A)):
            #    print(Q_A[i].y)
            if(pID == 0  and len(Q_A) > 1):
                Q_A.pop(pID)
            else:
                for i in range(pID-1, -1, -1):
                    Q_A.pop(i) 
            #if(len(Q_C) > 1):
            #    Q_C.pop(qID)
            #print("Q_A after")
            #for i in range(len(Q_A)):
            #    print(Q_A[i].y)

        return findBridge(Q_A, Q_C, iteration+1, debug=debug, hull=hull)



In [ ]:
def printTree(node, hull="lc"):
    node.printData()
    if(hull=="lc"):
        if(not node.lson is None):
            lson = node.lson
            printTree(lson, hull=hull)

        if(not node.rson is None):
            rson = node.rson
            printTree(rson, hull=hull)
    else:
        if(not node.rson is None):
            rson = node.rson
            printTree(rson, hull=hull)
            
        if(not node.lson is None):
            lson = node.lson
            printTree(lson, hull=hull)

In [ ]:
def DOWN(a, b, rootsList): #current Node a and target Node b 
    if(len(a.Q) == 1 and a.Q[0].y == b.Q[0].y):
        return rootsList
    else:
        QRson = []
        QLson = []
        for i in range(len(a.Q)):
            if(a.Q[i].y > a.maxY):
                QRson.append(a.Q[i])
            else:
                QLson.append(a.Q[i])
         
        for i in range(len(QRson)):
            a.rson.Q.insert(0,QRson[i])
        a.lson.Q.extend(QLson)
        
        a.Q.clear()
        
        for i in range(len(rootsList)-1,-1,-1):
            if(rootsList[i] == a):
                rootsList.pop(i)
        
        a.lson.B = 0
        a.rson.B = 0
        
        a.lson.f = None
        a.rson.f = None
        
        rootsList.append(a.lson)
        rootsList.append(a.rson)
        
        
        if(b.Q[0].y <= a.maxY):
            return DOWN(a.lson, b, rootsList)
        else:
            return DOWN(a.rson, b, rootsList)

In [ ]:
def DOWNperfect(a, b): #current Node a and target Node b 
    if(len(a.Q) == 1 and a.Q[0].y == b.Q[0].y):
        return
    else:
        QRson = []
        QLson = []
        for i in range(len(a.Q)):
            if(a.Q[i].y > a.maxY):
                QRson.append(a.Q[i])
            else:
                QLson.append(a.Q[i])
         
        for i in range(len(QRson)):
            a.rson.Q.insert(0,QRson[i])
        a.lson.Q.extend(QLson)
        
        a.Q.clear()
        
        
        a.lson.B = 0
        a.rson.B = 0
               
        
        if(b.Q[0].y <= a.maxY):
            return DOWN(a.lson, b)
        else:
            return DOWN(a.rson, b)

In [ ]:
def UP(a):
    QLson = a.lson.Q
    QRson = a.rson.Q
    p, q = findBridge(QLson, QRson, fig, 0, draw=False, debug=False, hull="lc")
    
    counter = 0
    popListQLson = []
    if(len(QLson) > 1):
        while(not QLson[counter].x == p[0] or not QLson[counter].y == p[1]):
            a.Q.append(QLson[counter])
            popListQLson.append(counter)
            counter += 1
        if(QLson[counter].x == p[0] and QLson[counter].y == p[1]):
            a.Q.append(QLson[counter])
            popListQLson.append(counter)
    else:
        a.Q.append(QLson[0])
        popListQLson.append(counter)

    popListQRson = []
    counter = 0
    if(len(QRson) > 1):
        while(not QRson[counter].x == q[0] or not QRson[counter].y == q[1]):
            counter += 1
        if(QRson[counter].x == q[0] and QRson[counter].y == q[1]):
            a.Q.append(QRson[counter])
            popListQRson.append(counter)
            counter += 1
        while(not counter == len(QRson)):
            a.Q.append(QRson[counter])
            popListQRson.append(counter)
            counter += 1
    else:
        a.Q.append(QRson[0])
        popListQRson.append(counter)

    a.lson.f = a
    a.rson.f = a


    a.lson.B = len(popListQLson)
    a.rson.B = len(popListQRson)

    for i in range(len(popListQLson)-1,-1,-1):
        a.lson.Q.pop(popListQLson[i])

    for i in range(len(popListQRson)-1,-1,-1):
        a.rson.Q.pop(popListQRson[i])

In [ ]:
def MYUP(rootsList, hull="lc"):
    n = len(rootsList)  
    
    if(n == 1):
        return rootsList
    opt1 = rootsList[n-1]
    opt2 = rootsList[n-2]
   
    if(False):
        print(hull)
        opt1.printData()
        if(opt1.lson):
            opt1.lson.printData()
        if(opt1.rson):
            opt1.rson.printData()
        opt2.printData()
        if(opt2.lson):
            opt2.lson.printData()
        if(opt2.rson):
            opt2.rson.printData()
        #printTree(opt1)
        #printTree(opt2)
    
    if(hull =="lc"):
        if(opt1.maxY >= opt2.maxY):
            Rson = opt1
            Lson = opt2
        else:
            Lson = opt1
            Rson = opt2
    else:
        if(opt1.maxY >= opt2.maxY):
            Lson = opt1
            Rson = opt2
        else:
            Rson = opt1
            Lson = opt2
    newIDD = ""
    if(hull == "lc"):
        newIDD += Lson.idd
        newIDD += Rson.idd
    else:
        newIDD += Rson.idd
        newIDD += Lson.idd
        
    a = InnerNode(idd=newIDD, f=None,  lson=Lson, rson=Rson, maxY = 0, Q=[], B=0)      
    QLson = a.lson.Q
    QRson = a.rson.Q
    
    if(hull == "lc"):
        p, q = findBridge(QRson, QLson, 0, draw=False, debug=False, hull=hull)
    else:
        p, q = findBridge(QLson, QRson, 0, draw=False, debug=False, hull=hull)        
    
    popListQLson = []
    popListQRson = []
    
    if(hull=="lc"):
        counter = 0
        if(len(QRson) > 1):
            while(not QRson[counter].x == p[0] or not QRson[counter].y == p[1]):
                a.Q.append(QRson[counter])
                popListQRson.append(counter)
                counter += 1
            if(QRson[counter].x == p[0] and QRson[counter].y == p[1]):
                a.Q.append(QRson[counter])
                popListQRson.append(counter)
        else:
            a.Q.append(QRson[0])
            popListQRson.append(counter)

        counter = 0
        if(len(QLson) > 1):
            while(not QLson[counter].x == q[0] or not QLson[counter].y == q[1]):
                counter += 1
            if(QLson[counter].x == q[0] and QLson[counter].y == q[1]):
                a.Q.append(QLson[counter])
                popListQLson.append(counter)
                counter += 1
            while(not counter == len(QLson)):
                a.Q.append(QLson[counter])
                popListQLson.append(counter)
                counter += 1
        else:
            a.Q.append(QLson[0])
            popListQLson.append(counter)
            
    else:
        counter = 0
        if(len(QLson) > 1):
            while(not QLson[counter].x == p[0] or not QLson[counter].y == p[1]):
                a.Q.append(QLson[counter])
                popListQLson.append(counter)
                counter += 1
            if(QLson[counter].x == p[0] and QLson[counter].y == p[1]):
                a.Q.append(QLson[counter])
                popListQLson.append(counter)
        else:
            a.Q.append(QLson[0])
            popListQLson.append(counter)

        counter = 0
        if(len(QRson) > 1):
            while(not QRson[counter].x == q[0] or not QRson[counter].y == q[1]):
                counter += 1
            if(QRson[counter].x == q[0] and QRson[counter].y == q[1]):
                a.Q.append(QRson[counter])
                popListQRson.append(counter)
                counter += 1
            while(not counter == len(QRson)):
                a.Q.append(QRson[counter])
                popListQRson.append(counter)
                counter += 1
        else:
            a.Q.append(QRson[0])
            popListQRson.append(counter)

    tempMaxY = 0
    if(hull == "lc"):
        if(len(a.lson.Q) > 0):
            for i in range(len(a.lson.Q)):
                if(a.lson.Q[i].y) > tempMaxY:
                       tempMaxY = a.lson.Q[i].y
        a.maxY = tempMaxY
    else:
        if(len(a.rson.Q) > 0):
            for i in range(len(a.rson.Q)):
                if(a.rson.Q[i].y) > tempMaxY:
                    tempMaxY = a.rson.Q[i].y
        a.maxY = tempMaxY  
             
    a.lson.f = a
    a.rson.f = a


    a.lson.B = len(popListQLson)
    a.rson.B = len(popListQRson)

    for i in range(len(popListQLson)-1,-1,-1):
        a.lson.Q.pop(popListQLson[i])

    for i in range(len(popListQRson)-1,-1,-1):
        a.rson.Q.pop(popListQRson[i])
        
    
    rootsList.pop(n-1)
    rootsList.pop(n-2)
    balancedTree = BALANCE(a, hull=hull)
    rootsList.append(balancedTree)

    return MYUP(rootsList, hull=hull)

In [ ]:
def INSERTDOWN(a, b, rootsList, hull="lc"): #current Node a and target Node b 
    if(len(a.Q) == 1):
        if(hull=="lc"):
            if(b.maxY <= a.maxY):
                rootsList.insert(len(rootsList)-1,b)
            else:
                rootsList.insert(len(rootsList),b)
        else:
            rootsList.insert(len(rootsList),b)
        return rootsList
    else:
        QRson = []
        QLson = []
        for i in range(len(a.Q)):
            if(hull=="lc"):
                if(a.Q[i].y > a.maxY):
                    QRson.append(a.Q[i])
                else:
                    QLson.append(a.Q[i])
            else:
                if(a.Q[i].y > a.maxY):
                    QLson.append(a.Q[i])
                else:
                    QRson.append(a.Q[i])
         
        if(hull=="lc"):
            for i in range(len(QRson)):
                a.rson.Q.insert(i,QRson[i])
            a.lson.Q.extend(QLson)
        else:
            for i in range(len(QLson)):
                a.lson.Q.insert(i,QLson[i])
            a.rson.Q.extend(QRson)
        
        a.Q.clear()
        
        for i in range(len(rootsList)-1,-1,-1):
            if(rootsList[i] == a):
                rootsList.pop(i)
        
        a.lson.B = 0
        a.rson.B = 0
        
        a.lson.f = None
        a.rson.f = None

        if(hull=="lc"):
            rootsList.append(a.lson)
            rootsList.append(a.rson)
        else:
            rootsList.append(a.rson)
            rootsList.append(a.lson)
        
        if(hull=="lc"):
            if(b.Q[0].y <= a.maxY):
                return INSERTDOWN(a.lson, b, rootsList, hull=hull)
            else:
                return INSERTDOWN(a.rson, b, rootsList, hull=hull)
        else:
            if(b.Q[0].y >= a.maxY):
                return INSERTDOWN(a.lson, b, rootsList, hull=hull)
            else:
                return INSERTDOWN(a.rson, b, rootsList, hull=hull)

In [ ]:
def DELETEDOWN(a, b, rootsList, hull="lc"): #current Node a and target point b 
    if(len(a.Q) == 1 and a.Q[0].y == b.y ):
        for i in range(len(rootsList)-1,-1,-1):
            if(rootsList[i].Q[0].y == b.y):
                rootsList.pop(i)       
        return rootsList
    else:
        QRson = []
        QLson = []
        for i in range(len(a.Q)):
            if(hull=="lc"):
                if(a.Q[i].y > a.maxY):
                    QRson.append(a.Q[i])
                else:
                    QLson.append(a.Q[i])
            else:
                if(a.Q[i].y > a.maxY):
                    QLson.append(a.Q[i])
                else:
                    QRson.append(a.Q[i])
                    
        if(hull == "lc"):
            for i in range(len(QRson)):
                a.rson.Q.insert(i,QRson[i])
            a.lson.Q.extend(QLson)
        else:
            for i in range(len(QLson)):
                a.lson.Q.insert(i,QLson[i])
            a.rson.Q.extend(QRson)

        a.Q.clear()
        
        for i in range(len(rootsList)-1,-1,-1):
            if(rootsList[i] == a):
                rootsList.pop(i)
        
        a.lson.B = 0
        a.rson.B = 0
        
        a.lson.f = None
        a.rson.f = None
                
        if(hull == "lc"):
            rootsList.append(a.lson)
            rootsList.append(a.rson)
        else:
            rootsList.append(a.rson)
            rootsList.append(a.lson)
            
        
        
        
        if(hull=="lc"):
            if(b.y <= a.maxY):
                return DELETEDOWN(a.lson, b, rootsList, hull=hull)
            else:
                return DELETEDOWN(a.rson, b, rootsList, hull=hull)
        else:
            if(b.y <= a.maxY):
                return DELETEDOWN(a.rson, b, rootsList, hull=hull)
                
            else:
                return DELETEDOWN(a.lson, b, rootsList, hull=hull)

In [ ]:
def drawHull(points, fig, hull=None,markPoint=None):
    n = len(points)
    
    figLength = len(fig.axes)
    axPoints = fig.add_subplot(figLength+1, 1, figLength+1)
    
    correctPlot(fig)

    for i in range(len(points)):
        #axPoints.scatter(points[i].x, points[i].y)      
        #drawPointColor(points[i].x, points[i].y, "b.", axPoints)
        drawPointColor(points[i].x, points[i].y, "bo", axPoints)
        if(hull):
            for j in range(len(hull)):
                #print(hull[j].x, hull[j].y)
                if(points[i].x == hull[j].x and points[i].y == hull[j].y):
                    axPoints.annotate(i, (points[i].x, points[i].y+3), fontsize=12)
    
    
    
    
    #for i in range(n):    
    #    drawPointColor(points[i].x, points[i].y, "b.", axPoints)
    if(markPoint):
        drawPointColor(markPoint.x, markPoint.y, "ro", axPoints)
    
    x0 = hull[0].x
    y0 = hull[0].y
    for p in hull:
            x1 = p.x
            y1 = p.y
            axPoints.plot([x0, x1], [y0, y1], linewidth=1.5)
            x0, y0 = x1, y1
    

In [ ]:
def getHeight(node):
    if(node.lson==None and node.rson==None):
        return 1
    else:
        lsonHeight = getHeight(node.lson)+1
        rsonHeight = getHeight(node.rson)+1
        if(lsonHeight >= rsonHeight):
            return lsonHeight
        else:
            return rsonHeight

In [ ]:
def recursiveTree(node):
    if(node.lson==None and node.rson==None):
        return node.maxY
    maxLson = recursiveTree(node.lson)
    maxRson = recursiveTree(node.rson)
    if(maxLson > maxRson):
        return maxLson
    else:
        return maxRson

In [ ]:
def calcMaxY(node,hull="lc"):
    if(hull=="lc"):
        return recursiveTree(node.lson)
    else:
        return recursiveTree(node.rson)

In [ ]:
def miniDOWN(a, hull="lc"):
    QRson = []
    QLson = []
    for i in range(len(a.Q)):
        if(hull=="lc"):
            if(a.Q[i].y > a.maxY):
                QRson.append(a.Q[i])
            else:
                QLson.append(a.Q[i])
        else:
            if(a.Q[i].y > a.maxY):
                QLson.append(a.Q[i])
            else:
                QRson.append(a.Q[i])

    if(hull == "lc"):
        for i in range(len(QRson)):
            a.rson.Q.insert(i,QRson[i])
        a.lson.Q.extend(QLson)
    else:
        for i in range(len(QLson)):
            a.lson.Q.insert(i,QLson[i])
        a.rson.Q.extend(QRson)


    a.Q.clear()
    
    a.lson.B = 0
    a.rson.B = 0
    
    return a.lson, a.rson

In [ ]:
def miniUP(a, hull="lc"):
    
    QLson = a.lson.Q
    QRson = a.rson.Q
    
    #print("QLSON")
    #for i in range(len(QLson)):
    #    print("({},{})".format(QLson[i].x, QLson[i].y))
    #print("QRSON")
    #for i in range(len(QRson)):
    #    print("({},{})".format(QRson[i].x, QRson[i].y))
    
    if(hull == "lc"):
        p, q = findBridge(QRson, QLson, 0, draw=False, debug=False, hull=hull)
    else:
        p, q = findBridge(QLson, QRson, 0, draw=False, debug=False, hull=hull)        
    
    #print("Bridge: ({},{}) - ({},{})".format(p[0],p[1],q[0],q[1]))
    popListQLson = []
    popListQRson = []
    
    if(hull=="lc"):
        counter = 0
        if(len(QRson) > 1):
            while(not QRson[counter].x == p[0] or not QRson[counter].y == p[1]):
                a.Q.append(QRson[counter])
                popListQRson.append(counter)
                counter += 1
            if(QRson[counter].x == p[0] and QRson[counter].y == p[1]):
                a.Q.append(QRson[counter])
                popListQRson.append(counter)
        else:
            a.Q.append(QRson[0])
            popListQRson.append(counter)

        counter = 0
        if(len(QLson) > 1):
            while(not QLson[counter].x == q[0] or not QLson[counter].y == q[1]):
                counter += 1
            if(QLson[counter].x == q[0] and QLson[counter].y == q[1]):
                a.Q.append(QLson[counter])
                popListQLson.append(counter)
                counter += 1
            while(not counter == len(QLson)):
                a.Q.append(QLson[counter])
                popListQLson.append(counter)
                counter += 1
        else:
            a.Q.append(QLson[0])
            popListQLson.append(counter)
            
    else:
        counter = 0
        if(len(QLson) > 1):
            while(not QLson[counter].x == p[0] or not QLson[counter].y == p[1]):
                a.Q.append(QLson[counter])
                popListQLson.append(counter)
                counter += 1
            if(QLson[counter].x == p[0] and QLson[counter].y == p[1]):
                a.Q.append(QLson[counter])
                popListQLson.append(counter)
        else:
            a.Q.append(QLson[0])
            popListQLson.append(counter)

        counter = 0
        if(len(QRson) > 1):
            while(not QRson[counter].x == q[0] or not QRson[counter].y == q[1]):
                counter += 1
            if(QRson[counter].x == q[0] and QRson[counter].y == q[1]):
                a.Q.append(QRson[counter])
                popListQRson.append(counter)
                counter += 1
            while(not counter == len(QRson)):
                a.Q.append(QRson[counter])
                popListQRson.append(counter)
                counter += 1
        else:
            a.Q.append(QRson[0])
            popListQRson.append(counter)    
        
    #print("Q_F")
    #for i in range(len(a.Q)):
    #    print(a.Q[i].y)
    
    a.lson.f = a
    a.rson.f = a


    a.lson.B = len(popListQLson)
    a.rson.B = len(popListQRson)

    for i in range(len(popListQLson)-1,-1,-1):
        a.lson.Q.pop(popListQLson[i])

    for i in range(len(popListQRson)-1,-1,-1):
        a.rson.Q.pop(popListQRson[i])
    
    return a

In [ ]:
def rotateRIGHT(node, hull="lc"):
    y, T4 = miniDOWN(node, hull=hull)
    x, T3 = miniDOWN(y, hull=hull)
    F = node.f
    
    y.rson = node
    T3.f = node
    node.lson = T3
    node.f = y  
    y.f = F

    node = miniUP(node, hull=hull)
    y = miniUP(y, hull=hull)
    
    node.maxY = calcMaxY(node, hull=hull)
    y.maxY = calcMaxY(y, hull=hull)
    
    node.B = abs(len(node.Q)-node.lson.B - node.rson.B)
    y.B = abs(len(y.Q)-y.lson.B - y.rson.B)

    if(F):
        if(F.lson == node):
            F.lson = y
        elif(F.rson == node):
            F.rson = y
    if(F):
        F.maxY = calcMaxY(F, hull=hull)
            
    return y

In [ ]:
def rotateLEFT(node, hull="lc"):
    T1, y = miniDOWN(node, hull=hull)
    T2, x = miniDOWN(y, hull=hull)
    F = node.f

    y.lson = node
    T2.f = node
    node.rson = T2
    node.f = y
    y.f = F

    node = miniUP(node, hull=hull)
    y = miniUP(y, hull=hull)
    
    
    node.maxY = calcMaxY(node, hull=hull)
    y.maxY = calcMaxY(y, hull=hull)
    
    node.B = abs(len(node.Q)-node.lson.B - node.rson.B)
    y.B = abs(len(y.Q)-y.lson.B - y.rson.B)
    
    if(F):
        if(F.lson == node):
            F.lson = y
        elif(F.rson == node):
            F.rson = y
    if(F):
        F.maxY = calcMaxY(F, hull=hull)
    return y

In [ ]:
def BALANCE(node, hull="lc"):   
    heightZ = getHeight(node)
    
    if(heightZ > 2):
        
        heightLSON = getHeight(node.lson)
        heightRSON = getHeight(node.rson)
        
        if(heightLSON-1 > heightRSON):
            y = node.lson
            heightYLSON = getHeight(y.lson)
            heightYRSON = getHeight(y.rson)
            if(heightYLSON > heightYRSON):
                #print("LEFT LEFT")
                rotateRIGHT(node, hull=hull)
                return node.f
            else:
                #print("LEFT RIGHT")
                miniDOWN(node,hull=hull)
                rotateLEFT(node.lson, hull=hull)
                rotateRIGHT(node, hull=hull)
                return node.f
            
        if(heightRSON-1 > heightLSON):
            y = node.rson
            heightYLSON = getHeight(y.lson)
            heightYRSON = getHeight(y.rson)
            if(heightYLSON > heightYRSON):
                #print("RIGHT LEFT")
                miniDOWN(node,hull=hull)
                rotateRIGHT(node.rson, hull=hull)
                rotateLEFT(node, hull=hull)
                return node.f
            else:
                #print("RIGHT RIGHT")
                rotateLEFT(node, hull=hull)
                return node.f
        
    return node

In [ ]:
def pointsToNodes(points):
    n = len(points)
    yList = []
    for i in range(n):    
        yList.append(points[i].y)
    
    sortedByY = [x for _, x in sorted(zip(yList, points), reverse=False)]  #sort points according to their y-value
    
    
    #for i in range(len(sortedByY)):
    #    print("{}:  ({},{})".format(i,sortedByY[i].x, sortedByY[i].y))
    
    nodeID = 0
    
    baseNodeslc = []
    baseNodesrc = []
    
    for i in range(len(sortedByY)):
        nodelc = InnerNode("{}".format(i), f=None,  lson=None, rson=None, maxY = sortedByY[i].y, Q=[sortedByY[i]], B=0)
        noderc = InnerNode("{}".format(i), f=None,  lson=None, rson=None, maxY = sortedByY[i].y, Q=[sortedByY[i]], B=0)
        baseNodeslc.append(nodelc)
        baseNodesrc.append(noderc)
    
    return baseNodeslc, baseNodesrc, sortedByY

In [ ]:
def COMPDOWN(a, b, rootsList, hull="lc"): #current Node a and target point b 
    #if(len(a.Q) == 1):
    if(a.lson is None and a.rson is None):
        return rootsList, a
    else:
        QRson = []
        QLson = []
        for i in range(len(a.Q)):
            if(hull=="lc"):
                if(a.Q[i].y > a.maxY):
                    QRson.append(a.Q[i])
                else:
                    QLson.append(a.Q[i])
            else:
                if(a.Q[i].y > a.maxY):
                    QLson.append(a.Q[i])
                else:
                    QRson.append(a.Q[i])
         
        if(hull=="lc"):
            for i in range(len(QRson)):
                a.rson.Q.insert(i,QRson[i])
            a.lson.Q.extend(QLson)
        else:
            for i in range(len(QLson)):
                a.lson.Q.insert(i,QLson[i])
            a.rson.Q.extend(QRson)
        
        a.Q.clear()
        
        for i in range(len(rootsList)-1,-1,-1):
            if(rootsList[i] == a):
                rootsList.pop(i)
        
        a.lson.B = 0
        a.rson.B = 0
        
        a.lson.f = None
        a.rson.f = None

        if(hull=="lc"):
            if(a.lson.maxY > a.rson.maxY):
                rootsList.append(a.lson)
                rootsList.append(a.rson)
            else:
                rootsList.append(a.rson)
                rootsList.append(a.lson)
                
        else:
            if(a.lson.maxY < a.rson.maxY):
                rootsList.append(a.rson)
                rootsList.append(a.lson)
            else:
                rootsList.append(a.lson)
                rootsList.append(a.rson)
            
            
        
        if(hull=="lc"):
            if(b.y <= a.maxY):
                return COMPDOWN(a.lson, b, rootsList, hull=hull)
            else:
                return COMPDOWN(a.rson, b, rootsList, hull=hull)
        else:
            if(b.y >= a.maxY):
                return COMPDOWN(a.lson, b, rootsList, hull=hull)
            else:
                return COMPDOWN(a.rson, b, rootsList, hull=hull)

In [ ]:
def INSERT(b, rootsListlc, rootsListrc):
    blc = Point(b.x,b.y)
    brc = Point(b.x,b.y)
    newNodelc = InnerNode(idd="Z", f=None,  lson=None, rson=None, maxY = blc.y, Q=[blc], B=0)
    newNoderc = InnerNode(idd="Z", f=None,  lson=None, rson=None, maxY = brc.y, Q=[brc], B=0)
    
    rootsListlc, a = COMPDOWN(rootsListlc[0], blc, rootsListlc, hull="lc")
    
    if(b.y <= a.maxY):
        rootsListlc.insert(len(rootsListlc)-1,newNodelc)
    else:
        rootsListlc.insert(len(rootsListlc),newNodelc)
    
    rootsListrc, a = COMPDOWN(rootsListrc[0], brc, rootsListrc, hull="rc")
    
    
    rootsListrc.insert(len(rootsListrc), newNoderc)
    
    rootsListlc = MYUP(rootsListlc, hull="lc")
    rootsListrc = MYUP(rootsListrc, hull="rc")
    
    return rootsListlc, rootsListrc

In [ ]:
def DELETE(b, rootsListlc, rootsListrc):
    rootsListlc, alc = COMPDOWN(rootsListlc[0], b, rootsListlc, hull="lc")
    rootsListrc, arc = COMPDOWN(rootsListrc[0], b, rootsListrc, hull="rc")
    
    for i in range(len(rootsListlc)-1,-1,-1):
        if(rootsListlc[i].Q[0].y == b.y):
            rootsListlc.pop(i)
    
    for i in range(len(rootsListrc)-1,-1,-1):
        if(rootsListrc[i].Q[0].y == b.y):
            rootsListrc.pop(i)
            
    rootsListlc = MYUP(rootsListlc, hull="lc")
    rootsListrc = MYUP(rootsListrc, hull="rc")
    
    return rootsListlc, rootsListrc

In [ ]:
def getHull(rootsListlc, rootsListrc):
    finalHull = []
    for i in range(len(rootsListlc[0].Q)):
        finalHull.append(Point(rootsListlc[0].Q[i].x,rootsListlc[0].Q[i].y))

    for i in range(len(rootsListrc[0].Q)-1,-1,-1):
        finalHull.append(Point(rootsListrc[0].Q[i].x,rootsListrc[0].Q[i].y))
        
    return finalHull

In [ ]:
#Generate Initial Set
n = 1000
#The code can handle up to 1000 points without problems. Any higher numbers have a chance to trigger errors,
#because the overhead gets too big

fig = plt.figure(figsize=(20,10))
points = create_points_circle(n)
rootsListlc, rootsListrc, points = pointsToNodes(points)
rootsListlc = MYUP(rootsListlc, hull="lc")
rootsListrc = MYUP(rootsListrc, hull="rc")

finalHull = getHull(rootsListlc, rootsListrc)
drawHull(points,fig,hull=finalHull,markPoint=None)

In [ ]:
#INSERT new point

isDuplicate = False
newPoint = Point(0.0,0.0)


for i in range(len(points)):
    #print(points[i].y, newPoint.y)
    if(points[i].y == newPoint.y):
        print("Y-Coordinate already exists")
        isDuplicate = True
    
if(not isDuplicate):
    points.append(newPoint)
    rootsListlc, rootsListrc = INSERT(newPoint, rootsListlc, rootsListrc)

    finalHull = getHull(rootsListlc, rootsListrc)
    
    fig = plt.figure(figsize=(20,10))
    drawHull(points,fig,hull=finalHull,markPoint=newPoint)

In [ ]:
#DELETE a point
pointExists = False
#deletePoint = Point(0.0, 0.0)
deletePoint = points[1000]

for i in range(len(points)-1,-1,-1):
    if(points[i].x == deletePoint.x and points[i].y == deletePoint.y):
        points.pop(i)
        pointExists = True
        
if(pointExists):
    rootsListlc, rootsListrc = DELETE(deletePoint, rootsListlc, rootsListrc)

    finalHull = getHull(rootsListlc, rootsListrc)
    fig = plt.figure(figsize=(20,10))
    drawHull(points,fig,hull=finalHull,markPoint=deletePoint)
else:
    print("Point does not exist")